<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d68702799728074d4b27802f13f2a5cca4074745eecd6d4d1b86a9da18551819
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-11 11:17:33
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.42 C
-------------------
Today PnL: 42.06 K (0.3%)
Current PnL: -17.49 L (-11.76%)
CY Booked + Current PnL: -6.46 L (-4.35%)
-------------------
Total profit:  2.10 L
Total loss:  -19.59 L
-------------------
Total Booked + Current PnL: 20.52 L (16.86%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.78%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.14 L (54.43%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.09,61.0,H-LC,11.21,173635.0,17458.0,8630.0,-0.46,11.18,4.97,16.71,22.0,2.02,1.33,30.57,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,61.0,M-LC,4.29,218188.0,16708.0,10517.0,0.63,8.29,4.82,13.51,60.0,1.59,1.67,21.93,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,107.47,65.0,M-SC,5.25,89401.0,-11376.0,11470.0,0.86,-11.29,12.83,0.09,245.0,-0.99,0.68,17.45,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114750.0,14246.0,16398.0,3.53,14.17,14.29,30.49,79.0,0.87,0.88,51.17,MH,ATH,METALS
41,ITC,452.00,-37.08,57.0,H-LC,3.01,203351.0,3213.0,17671.0,0.68,1.61,8.69,10.44,4.0,0.18,1.56,7.26,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.99,226443.0,1481.0,166164.0,1.37,0.66,73.38,74.53,192.0,0.01,1.73,33.12,XY24,BTT,STEEL
4,ANGELONE,3033.00,7.14,39.0,H-SC,6.93,190076.0,-930.0,58638.0,-0.80,-0.49,30.85,30.21,157.0,-0.02,1.45,20.29,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-21.00,42.0,H-MC,1.97,135996.0,208.0,42540.0,-0.08,0.15,31.28,31.48,107.0,0.00,1.04,12.29,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.67,245731.0,-2000.0,77061.0,-0.01,-0.81,31.36,30.30,92.0,-0.03,1.88,12.38,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.34,44.0,H-MC,3.23,137078.0,945.0,30185.0,-0.15,0.69,22.02,22.86,91.0,0.03,1.05,13.39,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303337.0,-12505.0,125157.0,-0.07,-3.96,41.26,35.67,5.0,-0.10,2.32,9.35,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.31,44.0,H-LC,13.63,218260.0,854.0,52644.0,0.56,0.39,24.12,24.61,48.0,0.02,1.67,55.43,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.37,45.0,H-LC,8.18,172286.0,-9473.0,44725.0,0.13,-5.21,25.96,19.39,86.0,-0.21,1.32,2.51,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.17,229895.0,-12029.0,75291.0,0.03,-4.97,32.75,26.15,8.0,-0.16,1.76,8.64,X40,ATH,IT
56,RELIANCE,1533.00,-13.66,49.0,H-LC,6.30,215904.0,5538.0,23253.0,0.51,2.63,10.77,13.68,37.0,0.24,1.65,19.59,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.62,55.0,H-LC,1.28,236182.0,-9969.0,89182.0,-1.10,-4.05,37.76,32.18,16.0,-0.11,1.81,30.53,X200,ATH,IT
41,ITC,452.00,-37.08,57.0,H-LC,3.01,203351.0,3213.0,17671.0,0.68,1.61,8.69,10.44,4.0,0.18,1.56,7.26,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.20,63.0,H-LC,3.34,285410.0,19984.0,36818.0,0.18,7.53,12.90,21.40,11.0,0.54,2.18,15.90,XY25,NTT,FMCG
20,CIPLA,1795.00,-20.12,51.0,H-LC,5.94,213446.0,8946.0,32465.0,-0.37,4.37,15.21,20.25,10.0,0.28,1.63,13.49,X40N,BTT,PHARMA
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303337.0,-12505.0,125157.0,-0.07,-3.96,41.26,35.67,5.0,-0.10,2.32,9.35,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.00,241.09,75.0,M-SC,6.29,152280.0,-30798.0,109779.0,14.39,-16.82,72.09,43.14,230.0,-0.28,1.17,43.24,XY24,NTT,MISC
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114750.0,14246.0,16398.0,3.53,14.17,14.29,30.49,79.0,0.87,0.88,51.17,MH,ATH,METALS
57,REPCOHOME,880.00,-60.51,51.0,H-SC,7.96,146942.0,-58056.0,203294.0,2.20,-28.32,138.35,70.85,134.0,-0.29,1.12,20.72,XY24,NTT,FINANCE
9,AWL,485.00,-64.90,54.0,H-MC,5.10,215902.0,-52545.0,184229.0,2.13,-19.57,85.33,49.05,116.0,-0.29,1.65,11.01,XY24,NTT,FMCG
12,BANKINDIA,190.00,-29.19,67.0,H-MC,5.53,183945.0,4137.0,108270.0,1.80,2.30,58.86,62.52,88.0,0.04,1.41,37.17,XR,NTT,BANKS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.0,-324.58,54.0,H-SC,8.41,95550.0,-37736.0,122333.0,-2.61,-28.31,128.03,63.47,139.0,-0.31,0.73,60.17,XR,NTT,CHEMICALS
67,SONACOMS,1006.0,-38.14,44.0,M-SC,11.97,77018.0,-24141.0,99030.0,-2.21,-23.86,128.58,74.03,202.0,-0.24,0.59,8.60,AR,BTT,AUTO
78,UJJIVANSFB,60.0,104.20,67.0,H-SC,12.71,128709.0,-13770.0,33297.0,-1.41,-9.66,25.87,13.70,163.0,-0.41,0.98,51.91,OX40N,NTT,BANKS
39,INFY,2275.0,-17.44,54.0,H-LC,9.80,322077.0,9189.0,162488.0,-1.34,2.94,50.45,54.87,3.0,0.06,2.46,9.78,X40,BTT,IT
86,WIPRO,420.0,-11.07,55.0,M-LC,6.28,156649.0,5704.0,103749.0,-1.21,3.78,66.23,72.51,53.0,0.05,1.20,10.03,XR,NTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,104.20,67.0,H-SC,12.71,128709.0,-13770.0,33297.0,-1.41,-9.66,25.87,13.70,163.0,-0.41,0.98,51.91,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,107.47,65.0,M-SC,5.25,89401.0,-11376.0,11470.0,0.86,-11.29,12.83,0.09,245.0,-0.99,0.68,17.45,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,55.0,H-SC,2.31,220572.0,-49095.0,85428.0,0.32,-18.21,38.73,13.47,138.0,-0.57,1.69,11.56,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,115.96,52.0,M-SC,2.75,141645.0,-32914.0,32947.0,0.95,-18.86,23.26,0.02,221.0,-1.00,1.08,23.21,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.23,114048.0,-29755.0,56499.0,-0.60,-20.69,49.54,18.60,149.0,-0.53,0.87,25.86,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,53.0,H-LC,13.71,293045.0,-52911.0,176765.0,0.24,-15.29,60.32,35.80,1.0,-0.30,2.24,3.81,X40,BTT,IT
39,INFY,2275.00,-17.44,54.0,H-LC,9.80,322077.0,9189.0,162488.0,-1.34,2.94,50.45,54.87,3.0,0.06,2.46,9.78,X40,BTT,IT
41,ITC,452.00,-37.08,57.0,H-LC,3.01,203351.0,3213.0,17671.0,0.68,1.61,8.69,10.44,4.0,0.18,1.56,7.26,X40,NTT,FMCG
83,VBL,671.64,-15.83,39.0,H-LC,6.30,303337.0,-12505.0,125157.0,-0.07,-3.96,41.26,35.67,5.0,-0.10,2.32,9.35,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,54.0,H-LC,12.27,251040.0,-10579.0,129788.0,1.32,-4.04,51.70,45.57,7.0,-0.08,1.92,6.82,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.99,226443.0,1481.0,166164.0,1.37,0.66,73.38,74.53,192.0,0.01,1.73,33.12,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.74,188840.0,24170.0,43792.0,-0.07,14.68,23.19,41.28,89.0,0.55,1.44,23.83,X40N,ATH,AC
84,VOLTAS,1918.49,-0.94,53.0,H-MC,3.17,207750.0,16008.0,80025.0,-0.71,8.35,38.52,50.08,99.0,0.20,1.59,15.71,XY25,ATH,AC
24,DABUR,735.00,-2.58,60.0,H-MC,6.00,212004.0,14976.0,74647.0,-0.71,7.60,35.21,45.49,102.0,0.20,1.62,21.63,XY24,BTT,FMCG
18,CAMS,5211.76,-3.83,55.0,H-SC,4.74,109449.0,7445.0,36479.0,0.46,7.30,33.33,43.06,122.0,0.20,0.84,25.66,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114750.0,14246.0,16398.0,3.53,14.17,14.29,30.49,79.0,0.87,0.88,51.17,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.53,60.0,M-SC,1.86,102165.0,10666.0,68083.0,-0.55,11.66,66.64,86.07,223.0,0.16,0.78,48.36,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.16,56.0,H-SC,12.20,138982.0,15646.0,118121.0,1.78,12.69,84.99,108.45,119.0,0.13,1.06,37.87,AR,ATH,MISC
86,WIPRO,420.00,-11.07,55.0,M-LC,6.28,156649.0,5704.0,103749.0,-1.21,3.78,66.23,72.51,53.0,0.05,1.20,10.03,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,67.0,H-MC,5.53,183945.0,4137.0,108270.0,1.80,2.30,58.86,62.52,88.0,0.04,1.41,37.17,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,43.49,89472.0,-24077.0,64125.0,-0.11,-21.20,71.67,35.27,268.0,-0.38,0.68,118.98,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.50,49.0,H-SC,14.36,89870.0,-11093.0,105624.0,1.02,-10.99,117.53,93.63,148.0,-0.11,0.69,33.86,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,57.0,H-SC,13.07,95055.0,-2925.0,24524.0,0.78,-2.99,25.80,22.05,152.0,-0.12,0.73,38.17,XR,ATH,FINANCE
82,VALIANTORG,838.00,-324.58,54.0,H-SC,8.41,95550.0,-37736.0,122333.0,-2.61,-28.31,128.03,63.47,139.0,-0.31,0.73,60.17,XR,NTT,CHEMICALS
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114750.0,14246.0,16398.0,3.53,14.17,14.29,30.49,79.0,0.87,0.88,51.17,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,43.49,89472.0,-24077.0,64125.0,-0.11,-21.20,71.67,35.27,268.0,-0.38,0.68,118.98,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-171.59,65.0,M-SC,3.01,82506.0,-19704.0,113091.0,0.56,-19.28,137.07,91.37,208.0,-0.17,0.63,33.93,XY25,NTT,FINANCE
75,TITAGARH,1548.00,-2.98,65.0,H-SC,6.46,159250.0,-32765.0,111650.0,0.13,-17.06,70.11,41.08,158.0,-0.29,1.22,34.94,XY24,NTT,ENGINEERING
49,NATIONALUM,247.44,-42.83,72.0,H-MC,3.53,114750.0,14246.0,16398.0,3.53,14.17,14.29,30.49,79.0,0.87,0.88,51.17,MH,ATH,METALS
8,ATULAUTO,844.00,3624.32,59.0,M-SC,7.46,116136.0,-31232.0,83049.0,-0.88,-21.19,71.51,35.16,236.0,-0.38,0.89,18.62,XY24,NTT,AUTO


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.27
1,25,44.35
2,50,75.16


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.85
LC    35.37
MC    23.74
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.83
X40      16.03
X40N     13.06
XY25     10.26
XR        9.51
AR        8.34
OX40N     7.72
MH        1.81
X200      1.81
X5K       1.46
SR        1.13
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.79
H-SC    24.92
H-MC    20.65
M-SC    14.28
M-LC     7.47
M-MC     2.76
L-SC     1.65
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.88,-15.79,74.41
FMCG,12.29,-3.31,38.33
FINANCE,9.05,-17.96,60.38
BANKS,7.82,-11.60,58.99
MISC,6.20,-13.55,78.63
PAINTS,6.16,-14.71,40.39
ELECTRICAL,5.86,-7.35,66.52
AC,3.78,1.99,31.20
AUTO,3.45,-16.68,69.30


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2695666.0,22
AR,1178420.0,9
XR,1052444.0,13
X40,872742.0,10
X40N,566936.0,10
OX40N,447358.0,10
XY25,410008.0,7
SR,196007.0,2
X5K,126271.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2764070.0,28
M-SC,1379095.0,17
H-LC,1363033.0,15
H-MC,1145542.0,15
M-LC,402471.0,5
M-MC,316188.0,2
L-SC,244585.0,3
L-MC,59706.0,1
L-LC,39080.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,910983.0,6
M-SC,XY24,795856.0,7
H-SC,AR,648896.0,5
H-LC,X40,581714.0,5
H-SC,XR,505479.0,6
H-MC,XY24,458061.0,4
H-LC,AR,368445.0,2
M-MC,XY24,316188.0,2
H-SC,X40N,247213.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
